Reforming the dataset "final_data.csv"

In [212]:
import pandas as pd
import numpy as np
from numpy import linalg as la
from numpy import random
from scipy.stats import norm
from scipy.stats import genextreme
from scipy import optimize
from scipy.io import loadmat
from tabulate import tabulate


Creating the path and loading the dataset

In [213]:
pd.options.display.float_format = '{:.9f}'.format
path = "/Users/nicholaihjelme/Desktop/mpak/"
# Final_DATA
choice_data = pd.read_csv(path + '/choice_data.csv', delimiter=';', encoding = 'unicode_escape')
df = choice_data
df.drop(['Shares'], axis=1,inplace=True)

Printing first 5 rows of our dataset

In [214]:
df.head()

,Year,Make-model-year-fuel,Make,Model,Variant,Size segment,Fuel-size segment,No. of registrations,Prices (2015-DKK),Ownership cost (DKK),Weight (kg),Fuel,Engine effect (kW),Horsepower,Size (m3),Footprint (m²),Cost/km (DKK),fuelPrice (DKK/unit)
0,2006,toyota-avensis-2006-Benzin,toyota,avensis,"1,8 STW MAN.",Small,BenzinSmall,5428,403349.299100000,346885.205500000,1820,Benzin,108.000000000,139.307692300,12.573440000,8.223261538,0.813669065,11.310000000
1,2006,volkswagen-passat-2006-Diesel,volkswagen,passat,"2,0 TDI",Large,DieselLarge,5378,378504.672900000,352803.014000000,2170,Diesel,103.000000000,128.333333300,12.734904000,8.657173611,0.681879195,10.160000000
2,2006,renault-clio-2006-Benzin,renault,clio,"1,2 8V 60 HK",Small,BenzinSmall,4518,134344.626200000,118899.032100000,1425,Benzin,43.000000000,60.000000000,8.872728000,6.248400000,0.677245509,11.310000000
3,2006,suzuki-swift-2006-Benzin,suzuki,swift,"1,5",Small,BenzinSmall,4406,163536.215000000,135059.819400000,1475,Benzin,68.000000000,94.000000000,9.379500000,6.253000000,0.734415584,11.310000000
4,2006,fiat-punto-2006-Benzin,fiat,punto,"1,2",Small,BenzinSmall,3743,150287.383200000,135188.422500000,1590,Benzin,48.000000000,63.888888890,9.483248000,6.392844444,0.689634146,11.310000000


Creating choices for each Fueltype

In [215]:
conditions = [
    (df['Fuel'] == 'EL'),
    (df['Fuel'] == 'Diesel')]
choices = [0, 1]
df['Fuelchoice'] = np.select(conditions, choices,default=2)
print(df['Fuelchoice'])

0       2
1       1
2       2
3       2
4       2
       ..
3377    1
3378    2
3379    2
3380    2
3381    2
Name: Fuelchoice, Length: 3382, dtype: int64


Number of observations for each year

In [216]:
year = 2006
for x in range(2006,2021):
    obs_count = df.loc[(df['Year'] == year)]
    print(len(obs_count))
    year += 1

186
210
215
198
217
225
232
242
248
246
246
239
234
230
214


Removing observations with the lowest number of registrations until there is the same number of observations for each year

In [217]:
year = df['Year'].max()
year

2020

In [218]:
year = 2006
for x in range(2006,2021):
     while len(df.loc[df['Year'] == year]) > len(df.loc[df['Year'] == 2006]):
          min_val = df.loc[(df['Year'] == year) & (df['Fuel'] != 'El'), 'No. of registrations'].idxmin()
          df.drop(index=min_val,inplace=True)
     year += 1


Number of observations for each year after deleting 

In [219]:
year = 2006
for x in range(2006,2021):
    obs_count = df.loc[(df['Year'] == year)]
    print(len(obs_count))
    year += 1

186
186
186
186
186
186
186
186
186
186
186
186
186
186
186


Creating a new column Shares based on each rows number. of registrations divided with the total number of registrations.

In [220]:
total_reg=df['No. of registrations'].sum(axis=0)
df["MarketShare"]= df["No. of registrations"]/total_reg
df.head()


,Year,Make-model-year-fuel,Make,Model,Variant,Size segment,Fuel-size segment,No. of registrations,Prices (2015-DKK),Ownership cost (DKK),Weight (kg),Fuel,Engine effect (kW),Horsepower,Size (m3),Footprint (m²),Cost/km (DKK),fuelPrice (DKK/unit),Fuelchoice,MarketShare
0,2006,toyota-avensis-2006-Benzin,toyota,avensis,"1,8 STW MAN.",Small,BenzinSmall,5428,403349.299100000,346885.205500000,1820,Benzin,108.000000000,139.307692300,12.573440000,8.223261538,0.813669065,11.310000000,2,0.001802671
1,2006,volkswagen-passat-2006-Diesel,volkswagen,passat,"2,0 TDI",Large,DieselLarge,5378,378504.672900000,352803.014000000,2170,Diesel,103.000000000,128.333333300,12.734904000,8.657173611,0.681879195,10.160000000,1,0.001786065
2,2006,renault-clio-2006-Benzin,renault,clio,"1,2 8V 60 HK",Small,BenzinSmall,4518,134344.626200000,118899.032100000,1425,Benzin,43.000000000,60.000000000,8.872728000,6.248400000,0.677245509,11.310000000,2,0.001500454
3,2006,suzuki-swift-2006-Benzin,suzuki,swift,"1,5",Small,BenzinSmall,4406,163536.215000000,135059.819400000,1475,Benzin,68.000000000,94.000000000,9.379500000,6.253000000,0.734415584,11.310000000,2,0.001463258
4,2006,fiat-punto-2006-Benzin,fiat,punto,"1,2",Small,BenzinSmall,3743,150287.383200000,135188.422500000,1590,Benzin,48.000000000,63.888888890,9.483248000,6.392844444,0.689634146,11.310000000,2,0.001243072


Creating a new column "Shares" based on each rows number. of registrations divided with the total number of registrations for each year.


In [221]:
year = 2006
df['market_year']=None
for x in range(2006,2021):
     total_reg_per_year = df.loc[(df['Year'] == year), 'No. of registrations'].sum(axis=0)
     share_year= ((df.loc[(df['Year'] == year), 'No. of registrations']) / total_reg_per_year)
     new_df = pd.DataFrame({'market_year': share_year})
     df.update(new_df)
     year += 1

In [223]:
df.to_csv(path + 'clogitchoice_data.csv')